In [21]:
from Bio import SeqIO
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle
import numpy as np
import torch
from torch.utils.data import Dataset
r_test_x = []
r_test_y = []
posit_1 = 1;
negat_0 = 0;

# define universe of possible input values
alphabet = 'OARNDCQEGHILKMFPSTWYV'
# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

i = 0
#-------------------------TEST DATASET----------------------------------------
#for positive sequence
def innertest1():
    #Input
    data = seq_record.seq
    #rint(data) 
    # integer encode input data
    for char in data:
        if char not in alphabet:
            return
    integer_encoded = [char_to_int[char] for char in data]
    r_test_x.append(integer_encoded)
    r_test_y.append(posit_1)
for seq_record in SeqIO.parse("./Datasets/independent_data/ecoli_test.fasta", "fasta"):
#for seq_record in SeqIO.parse("./Datasets/training_data/mus_train.fasta", "fasta"):
    innertest1()
    i += 1


#for negative sequence
def innertest2():
    #Input
    data = seq_record.seq
    print(data) 
    # integer encode input data
    for char in data:
        if char not in alphabet:
#             print(data, i)
            return
    integer_encoded = [char_to_int[char] for char in data]
    if integer_encoded[0] == 6: print(data, int_to_char[6], len(r_test_x))
    r_test_x.append(integer_encoded) 
    r_test_y.append(negat_0)
#for seq_record in SeqIO.parse("./Datasets/independent_data/mus_train_neg.fasta", "fasta"):
for seq_record in SeqIO.parse("./Datasets/independent_data/ecoli_test_neg.fasta", "fasta"):
    innertest2()
# Changing to array (matrix)    
r_test_x = np.array(r_test_x)
r_test_y = np.array(r_test_y)

# Balancing test dataset
# Testing Data Balancing by undersampling####################################
# rus = RandomUnderSampler(random_state=7)
# x_res3, y_res3 = rus.fit_resample(r_test_x, r_test_y)
# #Shuffling
# r_test_x, r_test_y = shuffle(x_res3, y_res3, random_state=7)
# r_test_x = np.array(r_test_x)
# r_test_y = np.array(r_test_y)

r_test_x = np.expand_dims(r_test_x, 1)
print(r_test_x.shape)
print(r_test_x)

LYTKYEKDDSIRKQRVKAVELFSLM
GIGVINFAYYLAKHGKRYSDGSANN
HMQIINEINTRFKTLVEKTWPGDEK
ILEANLLPPPEPKESWRRIMDELSV
TYFGELSRMTQFKDKSARYAENINA
KEILGDEADQYVKVPDTLDVWFDSG
AIQFAWELLTSEKWFALPKERLWVT
AALASVKGWVSAKLQOOOOOOOOOO
PIQETITFDDFAKVDLRVALIENAE
ERGKVIADYEERKAKIKADAEEAAR
RAALGVLRIIVEKNLNLDLQTLTEE
IRQIIDEDLASGKHTTVHTRFPPEP
KVISVQEMHAQIKOOOOOOOOOOOO
OOOOOOOOOMKIKTRFAPSPTGYLH
WDAAQSLLATYIKLNVARHQQGQPL
CGTAMDSYLIDPKRKLHVCGNNPTC
DAREAFLNITVTKDSRTRYSEAGHP
MATGSYPWIPPIKGSDTQDCFVYRT
EIYKRLIVSEDNKTLLGAVLVGDTS
TFIATSPMHIATKLRSTLDEVIERA
ISGLYERVPNIDKAIISVHTHDDLG
RNGLRPARYVITKDKLITCASEVGI
ADVIQIKVAQGAKPGEGGQLPGDKV
PAPCDYAITAIQKFLETDIPVFGIC
LPEQVRTNADLEKMVDTSDEWIVTR
LVDDERWARFNEKLENIERERQRLK
AAGDEVTVVRDEKKAREVALYRQGK
QLGIVSLREALEKAEEAGVDLVEIS
QLGIVSLREALEKAEEAGVDLVEIS Q 127
VGEDWISLDMHGKRPKAVNVRTAPH
OOOMSLLNVPAGKDLPEDIYVVIEI
OOOOOOOOOOOMKPYQRQFIEFALS
FGRPQRVAQEMQKEIALILQREIKD
AAALGQIEKQFGKGSIMRLGEDRSM
HTTIGKVDFDADKLKENLEALLVAL
KQYDINEAIALLKELATAKFVESVD
VRQHVIYKEAKIKOOOOOOOOOOOO
RAKYQRQLARAIKRARYLSLLPYTD
AFAVIV

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DLMal(nn.Module):
    def __init__(self):
        super(DLMal, self).__init__()
        self.embedding = nn.Embedding(25,21)
        self.conv1 = nn.Conv2d(1, 64, (15, 3))
        self.dropout1 = nn.Dropout(0.6)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.dropout2 = nn.Dropout(0.6)
        self.maxpool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4096, 768)
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(768, 256)
        self.dropout4 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.embedding(x)
        #print(x.shape)
        x = F.relu(self.conv1(x))
        #print(x.shape)
        x = self.dropout1(x)
        x = F.relu(self.conv2(x))
        #print(x.shape)
        x = self.dropout2(x)
        x = self.maxpool(x)
        #print(x.shape)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = F.relu(self.fc2(x))
        x = self.dropout4(x)
        return self.fc3(x)

In [23]:
PATH = './Path/dlmal_m_net.pth'
net = torch.load(PATH, map_location="cpu")
net.eval()

print(r_test_x[0])
print([int_to_char[i] for i in r_test_x[0][0]])
y = net(torch.from_numpy(np.array([r_test_x[0]])))
print(F.softmax(y, dim=1))

print(r_test_x[-1])
print([int_to_char[i] for i in r_test_x[-1][0]])
y = net(torch.from_numpy(np.array([r_test_x[-1]])))
print(F.softmax(y, dim=1))

TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(r_test_x)):
    y = net(torch.from_numpy(np.array([r_test_x[i]])))
    y = F.softmax(y, dim=1)
    y = y.detach().cpu().numpy()
    if (y[0][0] > y[0][1] and r_test_y[i] == 0):
        TN += 1
    elif (y[0][0] < y[0][1] and r_test_y[i] == 1):
        TP += 1
    elif r_test_y[i] == 0:
        FN += 1
        print([int_to_char[i] for i in r_test_x[i][0]], y)
    else:
        FP +=1
        print([int_to_char[i] for i in r_test_x[i][0]], y)
       
print("acc:", (TP+TN)/(TP+TN+FP+FN)*100)
print("sn:", (TP)/(TP+FN)*100)
print("sp:", TN/(TN+FP)*100)
print("mcc:", (TP*TN-FP*FN)/((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**(1/2))

[[11  7 13  1  6  3  1  1  2 11  8 18 12  1  7 12 20  4  1  2 11  9  9 10
  13]]
['L', 'E', 'M', 'A', 'Q', 'N', 'A', 'A', 'R', 'L', 'G', 'W', 'K', 'A', 'E', 'K', 'V', 'D', 'A', 'R', 'L', 'H', 'H', 'I', 'M']
tensor([[0.4768, 0.5232]], grad_fn=<SoftmaxBackward>)
[[14  7 10 20  3  3  7 16  4 15  2 14 12  7 19 18 17  7 19 14  6 10 13 12
   2]]
['F', 'E', 'I', 'V', 'N', 'N', 'E', 'S', 'D', 'P', 'R', 'F', 'K', 'E', 'Y', 'W', 'T', 'E', 'Y', 'F', 'Q', 'I', 'M', 'K', 'R']
tensor([[0.8321, 0.1679]], grad_fn=<SoftmaxBackward>)
['I', 'D', 'L', 'F', 'R', 'T', 'L', 'I', 'Q', 'A', 'V', 'A', 'K', 'V', 'T', 'G', 'A', 'T', 'D', 'L', 'S', 'N', 'K', 'S', 'L'] [[0.76452476 0.23547527]]
['A', 'A', 'A', 'N', 'K', 'R', 'V', 'S', 'N', 'I', 'L', 'A', 'K', 'S', 'D', 'E', 'V', 'L', 'S', 'D', 'R', 'V', 'N', 'A', 'S'] [[0.5012771 0.498723 ]]
['K', 'Y', 'V', 'L', 'A', 'R', 'T', 'D', 'T', 'A', 'T', 'D', 'K', 'D', 'Y', 'L', 'D', 'I', 'E', 'R', 'V', 'I', 'G', 'H', 'E'] [[0.6416199  0.35838005]]
['D', 'E', 'F', 'A', 'D'

['E', 'I', 'Y', 'K', 'R', 'L', 'I', 'V', 'S', 'E', 'D', 'N', 'K', 'T', 'L', 'L', 'G', 'A', 'V', 'L', 'V', 'G', 'D', 'T', 'S'] [[0.4339227 0.5660773]]
['A', 'D', 'V', 'I', 'Q', 'I', 'K', 'V', 'A', 'Q', 'G', 'A', 'K', 'P', 'G', 'E', 'G', 'G', 'Q', 'L', 'P', 'G', 'D', 'K', 'V'] [[0.38388747 0.6161126 ]]
['L', 'V', 'D', 'D', 'E', 'R', 'W', 'A', 'R', 'F', 'N', 'E', 'K', 'L', 'E', 'N', 'I', 'E', 'R', 'E', 'R', 'Q', 'R', 'L', 'K'] [[0.41932496 0.580675  ]]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'M', 'K', 'P', 'Y', 'Q', 'R', 'Q', 'F', 'I', 'E', 'F', 'A', 'L', 'S'] [[0.3146222  0.68537784]]
['F', 'G', 'R', 'P', 'Q', 'R', 'V', 'A', 'Q', 'E', 'M', 'Q', 'K', 'E', 'I', 'A', 'L', 'I', 'L', 'Q', 'R', 'E', 'I', 'K', 'D'] [[0.42036837 0.5796316 ]]
['A', 'A', 'A', 'L', 'G', 'Q', 'I', 'E', 'K', 'Q', 'F', 'G', 'K', 'G', 'S', 'I', 'M', 'R', 'L', 'G', 'E', 'D', 'R', 'S', 'M'] [[0.16856337 0.83143663]]
['H', 'T', 'T', 'I', 'G', 'K', 'V', 'D', 'F', 'D', 'A', 'D', 'K', 'L', 'K', 'E', 'N', 'L',